In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Veo 3 Advanced Controls

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo3_advanced_controls.ipynb">
      <img src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fvision%2Fgetting-started%2Fveo3_advanced_controls.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/vision/getting-started/veo3_advanced_controls.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo3_advanced_controls.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo3_advanced_controls.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo3_advanced_controls.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo3_advanced_controls.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo3_advanced_controls.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo3_advanced_controls.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| | |
|-|-|
|Author(s) | [Katie Nguyen](https://github.com/katiemn) |

## Overview

### Veo 3

Veo 3 on Vertex AI gives application developers access to Google's cutting-edge video generation. This model creates videos with stunning detail and realistic physics across a wide array of visual styles. Veo 3 enhances video quality from text and image prompts, and now includes dialogue and audio generation.

In this tutorial, you will learn how to use the Google Gen AI SDK for Python to interact with Veo 3.1 advanced control features, including:
  - Frame interpolation


## Get started

### Install Google Gen AI SDK for Python

In [2]:
%pip install --upgrade --quiet google-genai

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.

In [3]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Import libraries

In [4]:
import os
import time
import urllib.request

from IPython.display import Video, display
from PIL import Image as PIL_Image
from google import genai
from google.genai import types
import matplotlib.image as img
import matplotlib.pyplot as plt

### Set Google Cloud project information and create client

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [5]:
# fmt: off
PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
# fmt: on
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Define a helper function to display media

In [6]:
def show_video(video):
    if isinstance(video, str):
        file_name = video.split("/")[-1]
        !gsutil cp {video} {file_name}
        display(Video(file_name, embed=True, width=600, height=400))
    else:
        with open("sample.mp4", "wb") as out_file:
            out_file.write(video)
        display(Video("sample.mp4", embed=True, width=600, height=400))

### Load the video generation model

In [7]:
video_model = "veo-3.1-generate-preview"

## Video generation with Veo 3.1 advanced controls

### Frame interpolation

With frame interpolation, you can specify the first and last frame of the video clip, and Veo will generate the rest.

#### Load the images

Download the first and last frames that you will use in the request. If you would like to use local images, simply comment out the `wget` command and edit the `first_frame` and `last_frame` variables.

**Note:** This feature tends to work best when the starting and ending frames are similar in nature.

In [ ]:
# Download first frame
!wget https://storage.googleapis.com/cloud-samples-data/generative-ai/image/cookies.png
# fmt: off
first_frame = "cookies.png"  # @param {type: 'string'}

# Download last frame
!wget https://storage.googleapis.com/cloud-samples-data/generative-ai/image/cookies-milk.png
last_frame = "cookies-milk.png"  # @param {type: 'string'}
# fmt: on

# Display the images
fig, axis = plt.subplots(1, 2, figsize=(12, 6))
axis[0].imshow(img.imread(first_frame))
axis[0].set_title("First Frame")
axis[1].imshow(img.imread(last_frame))
axis[1].set_title("Last Frame")
for ax in axis:
    ax.axis("off")
plt.show()

With Veo 3.1, you have the option to generate videos with audio from a text prompt and/or input image(s). In order to generate a video in the following sample, specify the following info:
- **Prompt:** A description of the video you would like to see. Since you're starting with images you can also leave the prompt field empty.
- **Aspect ratio:** Select either 16:9 or 9:16.
- **Resolution:** 720p or 1080p.
- **Number of videos:** Set this value to 1, 2, 3, or 4.
- **Video duration:** Can be 4, 6, or 8 seconds.
- **Person generation:** Set to `allow_adult` or `dont_allow`.
- **Audio generation:** Set `generate_audio` to True if you'd like audio included in your generated video.


**Safety:** All Veo videos include [SynthID](https://deepmind.google/technologies/synthid/), which embeds a digital watermark directly into the AI-generated video.

In [ ]:
# fmt: off
prompt = "a hand reaches in and places a glass of milk next to the plate of cookies"  # @param {type: 'string'}
# fmt: on

operation = client.models.generate_videos(
    model=video_model,
    prompt=prompt,
    image=types.Image.from_file(location=first_frame),
    config=types.GenerateVideosConfig(
        last_frame=types.Image.from_file(location=last_frame),
        aspect_ratio="9:16",
        resolution="1080p",
        number_of_videos=1,
        duration_seconds=8,
        person_generation="allow_adult",
        generate_audio=True,
    ),
)

while not operation.done:
    time.sleep(10)
    operation = client.operations.get(operation)
    print(operation)

if operation.response:
    show_video(operation.result.generated_videos[0].video.video_bytes)

In this next example, you'll continue to use first and last frames stored in Cloud Storage. However, in this request, you'll reference the images in Cloud Storage directly, rather than downloading them locally first. If you'd like to use a different Cloud Storage image, replace the URL and gcs_uri variables below.

In [ ]:
first_frame = PIL_Image.open(
    urllib.request.urlopen(
        "https://storage.googleapis.com/cloud-samples-data/generative-ai/image/forest-1.png"
    )
)
first_frame_gcs = "gs://cloud-samples-data/generative-ai/image/forest-1.png"

last_frame = PIL_Image.open(
    urllib.request.urlopen(
        "https://storage.googleapis.com/cloud-samples-data/generative-ai/image/forest-2.png"
    )
)
last_frame_gcs = "gs://cloud-samples-data/generative-ai/image/forest-2.png"

# Display the images
fig, axis = plt.subplots(1, 2, figsize=(18, 6))
axis[0].imshow(first_frame)
axis[1].imshow(last_frame)
for ax in axis:
    ax.axis("off")
plt.show()

Rather than output video_bytes in this section, you'll save your video to Cloud Storage. In order to accomplish this, set your Cloud Storage bucket location in `output_gcs`.


In [ ]:
# fmt: off
prompt = "the chipmunk climbs down the tree to sit next to the rabbit"  # @param {type: 'string'}
output_gcs = "gs://[your-bucket-path]"  # @param {type: 'string'}
# fmt: on

operation = client.models.generate_videos(
    model=video_model,
    prompt=prompt,
    image=types.Image(gcs_uri=first_frame_gcs, mime_type="image/png"),
    config=types.GenerateVideosConfig(
        last_frame=types.Image(gcs_uri=last_frame_gcs, mime_type="image/png"),
        output_gcs_uri=output_gcs,
        aspect_ratio="16:9",
        number_of_videos=1,
        duration_seconds=6,
        resolution="1080p",
        person_generation="allow_adult",
        generate_audio=True,
    ),
)

while not operation.done:
    time.sleep(10)
    operation = client.operations.get(operation)
    print(operation)

if operation.response:
    show_video(operation.result.generated_videos[0].video.uri)